In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
train_set = pd.read_csv('D:/projects/ai/poc/homework/training_data_20210128.csv')
test_set = pd.read_csv('D:/projects/ai/poc/homework/testing_data_20210128.csv')

In [4]:

# plt.figure(figsize=(12,9))
# ax1 = sns.kdeplot(train_set.balance,label='train_set')
# ax2 = sns.kdeplot(test_set.balance,label='test_set')
from scipy.stats import ks_2samp
ks_2samp(train_set.LAYER,test_set.LAYER)


KstestResult(statistic=0.2481175038597152, pvalue=2.9435323369789208e-55)

In [7]:
train_set['Is_Test'] = 0
test_set['Is_Test'] = 1
# 将 Train 和 Test 合成一个数据集。
df_adv = pd.concat([train_set, test_set])
# 将样本根据时间排序
train_set['DATA_DATE'] = pd.to_datetime(train_set['DATA_DATE'], infer_datetime_format=True)
test_set['DATA_DATE'] = pd.to_datetime(test_set['DATA_DATE'], infer_datetime_format=True)
train_set = train_set.sort_values('DATA_DATE').reset_index(drop=True)
train_set.drop(['DATA_DATE'], axis=1, inplace=True)


In [8]:
#4.1 人工划分验证集
# 前80%的样本作为训练集，后20%的样本作为验证集
df_validation_1 = train_set.iloc[int(0.8 * len(train_set)):, ]
df_train_1 = train_set.iloc[:int(0.8 * len(train_set)), ]


In [16]:
adv_data = lgb.Dataset(
    data=df_adv.drop('Is_Test', axis=1), label=df_adv.loc[:, 'Is_Test'])
# 定义模型参数
params = {
    'boosting_type': 'gbdt',
    'colsample_bytree': 1,
    'learning_rate': 0.1,
    'max_depth': 5,
    'min_child_samples': 100,
    'min_child_weight': 1,
    'min_split_gain': 0.0,
    'num_leaves': 20,
    'objective': 'binary',
    'random_state': 50,
    'subsample': 1.0,
    'subsample_freq': 0,
    'metric': 'auc',
    'num_threads': 8
}
categorical_columns=['DATA_DATE', 'LOT_ID', 'STATUS', 'CHIPNAME', 'LAYER', 'OP_NO', 'LOT_TYPE', 'WS_DATE', 'ACTUAL_WP_OUT']


def factor_data(df, col):
    df_labeled, _ = df[col].factorize(sort=True)
    # MAKE SMALLEST LABEL 1, RESERVE 0
    df_labeled += 1
    # MAKE NAN LARGEST LABEL
    df_labeled = np.where(df_labeled==0, df_labeled.max()+1, df_labeled)
    df[col] = df_labeled

for col in tqdm(categorical_columns):
    factor_data(df_all, col)


# 交叉验证
adv_cv_results = lgb.cv(
    params,
    adv_data,
    num_boost_round=10000,
    nfold=5,
    # categorical_feature=categorical_columns,
    early_stopping_rounds=200,
    verbose_eval=True,
    seed=42)

print('交叉验证中最优的AUC为 {:.5f}，对应的标准差为{:.5f}.'.format(
    adv_cv_results['auc-mean'][-1], adv_cv_results['auc-stdv'][-1]))

print('模型最优的迭代次数为{}.'.format(len(adv_cv_results['auc-mean'])))

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: DATA_DATE, LOT_ID, STATUS, CHIPNAME, LAYER, OP_NO, LOT_TYPE, WS_DATE, ACTUAL_WP_OUT

In [11]:
#4.2 和测试集最相似的样本作为验证集
# 通过抗验证中的模型，得到各个样本属于测试集的概率
import lightgbm as lgb
params['n_estimators'] = len(adv_cv_results['auc-mean'])

model_adv = lgb.LGBMClassifier(**params)
model_adv.fit(df_adv.drop('Is_Test', axis=1), df_adv.loc[:, 'Is_Test'])
preds_adv = model_adv.predict_proba(df_adv.drop('Is_Test', axis=1))[:, 1]

# 只需要训练样本的概率
train_set_copy = train_set.copy()
train_set_copy['is_test_prob'] = preds_adv[:len(train_set)]

# 根据概率排序
train_set_copy = train_set_copy.sort_values('is_test_prob').reset_index(drop=True)

# 将概率最大的20%作为验证集
df_validation_2 = train_set_copy.iloc[int(0.8 * len(train_set)):, ]
train_set_2 = train_set_copy.iloc[:int(0.8 * len(train_set)), ]

NameError: name 'adv_cv_results' is not defined